In [2]:
import os
import pandas as pd
import sqlite3
from datetime import datetime as dt
print(os.getcwd())

c:\Users\rafae\PIPELINE\projeto_pipeline


In [4]:
diretorio_db = 'c:/Users/rafae/PIPELINE/projeto_pipeline/pipeline'
os.makedirs(diretorio_db, exist_ok=True)  # Garante que a pasta existe
caminho_db = os.path.join(diretorio_db, 'pipeline.db')

conn = sqlite3.connect(caminho_db)
print("Conexão com banco de dados estabelecida")


Conexão com banco de dados estabelecida


CARREGAR DADOS LIMPOS (SILVER)

In [13]:
df_limpo = pd.read_csv('c:/Users/rafae/PIPELINE/projeto_pipeline/silver/twitter_sentment.csv')

# Salvar df_limpo no banco de dados SQLite em uma tabela chamada 'tabela_principal'
df_limpo.to_sql('tabela_principal', conn, if_exists='replace', index=False)
print(f"Tabela principal criada: {len(df_limpo)} registros")


Tabela principal criada: 1599999 registros


CARREGAR DADOS AGREGADOS (GOLD)

In [14]:

# Carregar os CSVs das agregações
metricas_diarias = pd.read_csv('c:/Users/rafae/PIPELINE/projeto_pipeline/gold/metricas_diarias.csv')
identificador_mencoes = pd.read_csv('c:/Users/rafae/PIPELINE/projeto_pipeline/gold/identificador_mencoes.csv')
analise_usuarios = pd.read_csv('c:/Users/rafae/PIPELINE/projeto_pipeline/gold/analise_usuarios.csv')

# Salvar no banco
metricas_diarias.to_sql('metricas_diarias', conn, if_exists='replace', index=False)
identificador_mencoes.to_sql('identificador_mencoes', conn, if_exists='replace', index=False)
analise_usuarios.to_sql('analise_usuarios', conn, if_exists='replace', index=False)

print("Agregações carregadas no banco SQLite.")


Agregações carregadas no banco SQLite.


In [ ]:


# Renomear colunas para evitar conflitos (exemplo: adicionar prefixo)
metricas_diarias = metricas_diarias.add_prefix('metrica_')
identificador_mencoes = identificador_mencoes.add_prefix('mencoes_')
analise_usuarios = analise_usuarios.add_prefix('analise_')

# Concatenar os DataFrames lado a lado
df_unico = pd.concat([ metricas_diarias, identificador_mencoes, analise_usuarios], axis=1)

# Salvar o DataFrame único em um CSV
df_unico.to_csv('c:/Users/rafae/PIPELINE/projeto_pipeline/gold/todas_agregacoes.csv', index=False)


In [18]:
df_unico = pd.read_csv("c:/Users/rafae/PIPELINE/projeto_pipeline/gold/todas_agregacoes.csv")

C:\Users\rafae\AppData\Local\Temp\ipykernel_14352\31018338.py:1: DtypeWarning: Columns (3,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_unico = pd.read_csv("c:/Users/rafae/PIPELINE/projeto_pipeline/gold/todas_agregacoes.csv")


In [ ]:
df_gold = pd.read_csv("c:/Users/rafae/PIPELINE/projeto_pipeline/gold/todas_agregacoes.csv")


C:\Users\rafae\AppData\Local\Temp\ipykernel_14352\1205380990.py:1: DtypeWarning: Columns (3,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_gold = pd.read_csv("c:/Users/rafae/PIPELINE/projeto_pipeline/gold/todas_agregacoes.csv")


In [24]:
print(df_limpo.columns)

df_limpo = df_limpo.rename(columns={"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D" : "texto"})


Index(['indice', 'user_id', 'data', 'consulta', 'usuario',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D',
       'data_ingestao', 'fonte_arquivo', 'data_processamento'],
      dtype='object')


In [27]:
print(df_limpo.columns)

Index(['indice', 'user_id', 'data', 'consulta', 'usuario', 'texto',
       'data_ingestao', 'fonte_arquivo', 'data_processamento'],
      dtype='object')


CRIAR TABELAS RELACIONADAS

In [28]:
# Extrair informações únicas dos usuários
usuarios = df_limpo[['usuario', 'user_id']].drop_duplicates()
usuarios.to_sql('usuarios', conn, if_exists='replace', index=False)


1598314

In [31]:
postagens = df_limpo[["data_ingestao", "texto"]].drop_duplicates()
postagens.to_sql('postagens', conn, if_exists='replace', index=False)

1581465

 VERIFICAR TABELAS CRIADAS


In [34]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tabelas = cursor.fetchall()
print("\nTabelas no banco de dados:")

for tabela in tabelas:
    print(f" - {tabela[0]}")

conn.close()
print("\nDados carregados com sucesso no banco de dados!")


Tabelas no banco de dados:
 - tabela_principal
 - metricas_diarias
 - identificador_mencoes
 - analise_usuarios
 - usuarios
 - postagens

Dados carregados com sucesso no banco de dados!
